In [ ]:
from FasterRCNN import *

In [ ]:
#  Build faster rcnn model
img_input, shared_layers, roi_input = build_base_model()
rpn_class, rpn_position = build_rpn_layer(shared_layers)
rpn_class_model = Model(img_input, rpn_class)
rpn_positoin_model = Model(img_input, rpn_position)
vgg_model = Model(img_input, shared_layers)

In [ ]:
# compile the models
model_rpn, model_classifier, model_all = faster_rcnn(shared_layers, roi_input, img_input)
model_rpn.compile(optimizer='adam', loss=[rpn_loss_cls(9), rpn_loss_regr(9)])
model_classifier.compile(optimizer='adam',
                         loss=[class_loss_cls, class_loss_regr(37)],
                         metrics={'dense_class_{}'.format(38): 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')

In [ ]:
# hyperparameters and settings
epoch_length = 1
num_epochs = 200
iter_num = 0
losses = np.zeros((epoch_length, 5))
num_rois = 4
best_loss = float('inf')
output_model_name = "weight/trash_model"

In [ ]:
# load the dataset
input_img_data = np.load('bacup/test_input.npy', allow_pickle=True)
label_data = np.load('bacup/test_output.npy', allow_pickle=True)

In [ ]:
# Training
for epoch_num in range(num_epochs):
    rpn_accuracy_for_epoch = []
    for img, annotation in zip(input_img_data, label_data):
        # Generate X (x_img) and label Y ([y_rpn_cls, y_rpn_regr])
        X, Y = get_anchor_gt([(img, annotation)])

        loss_rpn = model_rpn.train_on_batch(X, Y)
        P_rpn = model_rpn.predict_on_batch(X)
        # R: bboxes (shape=(300,4))
        # Convert rpn layer to roi bboxes
        R = rpn_to_roi(P_rpn[0], P_rpn[1], use_regr=True, overlap_thresh=0.9, max_boxes=300)
        X2, Y1, Y2, IouS = calc_iou(R, annotation, classes_count=38)

        # If X2 is None means there are no matching bboxes
        if X2 is None:
            rpn_accuracy_for_epoch.append(0)
            continue

        # Find out the positive anchors and negative anchors
        neg_samples = np.where(Y1[0, :, -1] == 1)
        pos_samples = np.where(Y1[0, :, -1] == 0)

        if len(neg_samples) > 0:
            neg_samples = neg_samples[0]
        else:
            neg_samples = []

        if len(pos_samples) > 0:
            pos_samples = pos_samples[0]
        else:
            pos_samples = []

        rpn_accuracy_for_epoch.append((len(pos_samples)))

        if num_rois > 1:
            # If number of positive anchors is larger than 4//2 = 2, randomly choose 2 pos samples
            if len(pos_samples) < num_rois // 2:
                selected_pos_samples = pos_samples.tolist()
            else:
                selected_pos_samples = np.random.choice(pos_samples, num_rois // 2, replace=False).tolist()

            # Randomly choose (num_rois - num_pos) neg samples
            try:
                selected_neg_samples = np.random.choice(neg_samples, num_rois - len(selected_pos_samples),
                                                        replace=False).tolist()
            except:
                selected_neg_samples = np.random.choice(neg_samples, num_rois - len(selected_pos_samples),
                                                        replace=True).tolist()

            # Save all the pos and neg samples in sel_samples
            sel_samples = selected_pos_samples + selected_neg_samples
        else:
            # in the extreme case where num_rois = 1, we pick a random pos or neg sample
            selected_pos_samples = pos_samples.tolist()
            selected_neg_samples = neg_samples.tolist()
            if np.random.randint(0, 2):
                sel_samples = random.choice(neg_samples)
            else:
                sel_samples = random.choice(pos_samples)

        loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]],
                                                     [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])

        losses[iter_num, 0] = loss_rpn[1]
        losses[iter_num, 1] = loss_rpn[2]

        losses[iter_num, 2] = loss_class[1]
        losses[iter_num, 3] = loss_class[2]
        losses[iter_num, 4] = loss_class[3]

    iter_num += 1
    print("Iter_num: {}".format(iter_num))

    if iter_num == epoch_length:
        loss_rpn_cls = np.mean(losses[:, 0])
        loss_rpn_regr = np.mean(losses[:, 1])
        loss_class_cls = np.mean(losses[:, 2])
        loss_class_regr = np.mean(losses[:, 3])
        class_acc = np.mean(losses[:, 4])

        mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
        rpn_accuracy_for_epoch = []

        curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
        iter_num = 0

        if curr_loss < best_loss:
            print('Total loss decreased from {} to {}, saving weights'.format(best_loss, curr_loss))
            best_loss = curr_loss
            model_all.save_weights(output_model_name)

        break
print('Training complete, exiting.')
exit(1)